# imdb dataset으로 감성 분류

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras import layers

In [ ]:
# encoder block
def transformer_encoder(inputs, head_size,num_heads,ff_dim,dropout=0):
    # Attention
    x= layers.LayerNormalization(epsilon=1e-6)(inputs) # sequence에 따른 고정길이 정규화로 BatchNormalization에 비해 언어 모델에 유연?
    x=layers.MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(x, x)
    res = x+inputs

    # FeedFprward NeuralNet
    x=layers.LayerNormalization(epsilon=1e-6)(res)
    x=layers.Conv1D(filters=ff_dim,kernel_size=1,activation='relu')(x)
    x=layers.Dropout(dropout)(x)
    x=layers.Conv1D(filters=inputs.shape[-1],kernel_size=1)(x)
    return x+res

def build_sentment_model(input_shape,head_size, num_heads, ff_dim, num_transformer_blocks,mlp_units,dropout=0,mlp_dropout=0):
    inputs=keras.Input(shape=input_shape)
    embedding_layer=layers.Embedding(input_dim=10000,output_dim=64,input_length=input_shape[0])(inputs)
    x=embedding_layer+tf.random.normal(shape=tf.shape(embedding_layer))
    # Transformer blocks
    for _ in range(num_transformer_blocks):
        x=transformer_encoder(x,head_size,num_heads,ff_dim,dropout)

    x = layers.GlobalAveragePooling1D()(x) # 공간차원축소

    # MLP
    for dim in mlp_units:
        x=layers.Dense(dim, activation='relu')(x)
        x=layers.Dropout(mlp_dropout)(x)

    outputs=layers.Dense(1,activation='sigmoid')(x)
    return keras.Model(inputs=inputs,outputs=outputs)


In [ ]:
(x_train, y_train),(x_val,y_val)= keras.datasets.imdb.load_data(num_words=10000)
x_train= keras.preprocessing.sequence.pad_sequences(x_train,maxlen=100)
x_val=keras.preprocessing.sequence.pad_sequences(x_val,maxlen=100)

# example usage
input_shape=(100,)
head_size=256
num_heads=4
ff_dim=4
num_transformer_blocks=4
mlp_units=[128]
dropout=0.25
mlp_dropout=0.25

model=build_sentment_model(
    input_shape,head_size, num_heads, ff_dim, num_transformer_blocks,mlp_units,dropout,mlp_dropout
    )

print(model.summary())

In [ ]:
model.compile(loss='binary_crossentropy',optimizer = keras.optimizers.Adam(learning_rate=1e-4),metrics=['acc'])

history = model.fit(x_train,y_train,validation_data=(x_val,y_val),epochs=10,batch_size=32,verbose=2)
print(history.history)


In [ ]:
from sre_constants import MAX_REPEAT
text_sample=["I love movie", "This movie sucks, It's so boring."]
max_len=100
tok=keras.datasets.imdb.get_word_index()
text_sequence=[[tok[word] if word in tok else 0 for word in sample.split()] for sample in text_sample]
text_sequence= keras.preprocessing.sequence.pad_sequences(text_sequence,maxlen=max_len)
# print(text_sequence)

pred = model.predict(text_sequence) # 이항분류가 나오면 logistic regresssion과 logic 변환을 떠올려야 한다., odds비와 logit 변환을 떠올려야 한다.

b_pred = (pred > 0.5).astype(int)

for i, samp in enumerate(text_sample):
    print(f"sample : {samp}")
    print(f"predicted : {'Positive' if b_pred[i]==1 else 'Negative'}")
    print(f"confidence : {pred[i][0]:.3f}\n")
